# Construir una Regresión Multivariada


La primera regresión que construimos fue una regresión univariada, lo que significa que teníamos una variable independiente, que en este caso era 'loan_amount' (Monto del préstamo). Nuestra regresión lineal demostró que, en general, cuantos más prestamistas mayor es el monto del préstamo. 

Ahora usemos las otras variables en nuestros datos y veamos cuáles tienen una fuerte relación lineal con loan_amount.

In [ ]:
# Cargando paquetes
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
plt.rcParams['figure.figsize'] = (12, 8)
sns.set()
sns.set(font_scale=1.5)

# Paquetes para validar supuestos
from scipy import stats as stats
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, make_scorer
#import statsmodels.formula.api as sm
import statsmodels.api as sm

2) Carga de Data
---

In [ ]:
# Cargamos los datos guardados de forma local
path = '../data/'
filename = 'loans.csv'
df = pd.read_csv(path+filename)

In [ ]:
# Alternativamente podemos obtener la data clonando el repositorio de Delta Analytics
!git clone https://github.com/DeltaAnalytics/machine_learning_for_good_data
df = pd.read_csv("machine_learning_for_good_data/loans.csv")

In [ ]:
df.columns

In [ ]:
df.columns.tolist()

In [ ]:
# exploramos nuestra data
df.head()

In [ ]:
# empezaremos con una regresion multivariada usando las variables numéricas como nuestras variables independientes
df.dtypes

In [ ]:
# grafico loan amount vs. lender count
ax = sns.regplot(x='lender_count', y='loan_amount', data=df, fit_reg=False)
ax.set_title('Scatter plot of lender count vs requested loan amount')

¿Cómo es la relación entre la cantidad de prestamistas (lender_count) y el monto del préstamo (loan_amount)? Es lineal?
<br>
<br>
<br>

In [ ]:
# Grafica loan amount vs. repayment term
ax = sns.regplot(x='repayment_term', y='loan_amount', data=df, fit_reg=False)
ax.set_title('Scatter plot of repayment term vs requested loan amount')

¿Cómo es la relación entre el plazo de reembolso (repayment) y el monto del préstamo (loan amount)? Es lineal?

<br>
<br>
<br>

In [ ]:
for sector in df['sector'].unique():
    ax = sns.regplot(x='lender_count', y='loan_amount', data=df[df['sector']==sector], fit_reg=False)
    ax.set_title(sector)
    plt.show()


La relación lineal entre lender_count y loan_amount es más fuerte para la salud (Health) y el uso personal (Personal Use).

In [ ]:
for country in df['location_country_code'].unique():
    ax = sns.lmplot(x='lender_count', y='loan_amount', data=df[df['location_country_code']==country], fit_reg=False,
                    hue = 'sector')
    ax = plt.gca()
    ax.set_title(country)


La relación lineal entre lender_count y loan_amount es más fuerte para el país ZM

In [ ]:
# Nuestra regresión lineal simple del notebook anterior
pd.options.mode.chained_assignment = None  # default='warn'

# Variable dependiente
y_column = 'loan_amount'
y = df[y_column]
# Variables independientes
x_columns = ['lender_count']
X = df[x_columns]
#Agregar un término intercepto a las variables independientes. 
#Esto es necesario para incluir el término constante de la ecuación de regresión lineal.
X['cnst'] = 1
# Dividimos entre training y test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model1 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model1.summary())

¿Qué cree que sucederá si incluimos la variable funded_amount, que sabemos que está altamente correlacionada con el recuento de prestamistas?

<br>
<br>
<br>

In [ ]:
# Variable dependiente
y_column = 'loan_amount'
y = df[y_column]
# Variables independientes
x_columns = ['lender_count', 'funded_amount']
X = df[x_columns]
#Agregar un término intercepto a las variables independientes. 
#Esto es necesario para incluir el término constante de la ecuación de regresión lineal.
X['cnst'] = 1
# Dividimos entre training y test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model2 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model2.summary())

Para mejorar nuestro modelo, podemos 1) eliminar variables altamente correlacionadas que están causando multicolinealidad, 2) incluir las variables categóricas que no hemos utilizado previamente (es decir, sector y código_país_de_ ubicación), y 3) incluir 'Interaction Terms'. Los términos de interacción son variables que describen la interacción entre dos o más variables independientes.

In [ ]:
for country in df['location_country_code'].unique():
    if country is not np.nan:
        df['country_'+country] = np.where(df.location_country_code == country, 1, 0)

In [ ]:
df.head()

Modelo Multivariado
===

Una variable que toma el valor de uno o cero dependiendo del valor de otra variable se llama "indicador". La variable country_ZM que creamos es 1 si la variable location_country_code es "ZM" y 0 en caso contrario. Estas variables indicadoras son cómo utilizamos las variables categóricas en la regresión. 

Realicemos una regresión lineal multivariante utilizando loan_amount como nuestra variable independiente, y lender_count, el indicador ZM del país y los indicadores de alimentos, uso personal y minorista de los sectores como nuestras variables dependientes.

$$Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_3 + \beta_4 X_4 + \beta_5 X_5$$

$\beta_0$ se llama intercepto y representa el valor medio esperado de Y cuando todas las variables explicativas son iguales a 0.  

$\beta_1$ se llama coeficiente beta y representa el cambio esperado en el valor de Y que resulta de un cambio de una unidad en $X_1$. En nuestro ejemplo, $X_1$ será lender_count.

$\beta_2$ se llama coeficiente beta y representa el cambio esperado en el valor de Y que resulta de un cambio de una unidad en $X_2$. En nuestro ejemplo, $X_2$ sera el indicador del pais ZM.

$\beta_3$ se llama coeficiente beta y representa el cambio esperado en el valor de Y que resulta de un cambio de una unidad en $X_3$. En nuestro ejemplo, $X_3$ será el indicador del sector Food.

$\beta_4$ llama coeficiente beta y representa el cambio esperado en el valor de Y que resulta de un cambio de una unidad en $X_4$. En nuestro ejemplo, $X_4$ será el indicador del sector Personal Use.

$\beta_5$ llama coeficiente beta y representa el cambio esperado en el valor de Y que resulta de un cambio de una unidad en $X_5$. En nuestro ejemplo, $X_5$ será el indicador del sector Retail.

In [ ]:
y = df['loan_amount']
X = df[['lender_count', 'country_RW']]

# Add an intercept term to the independent variables

X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model3 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model3.summary())

Aumentamos nuestro R cuadrado ajustado de 0.811 a 0.846. También mirar los p-values nos dice que todos los coeficientes son significativos. Es posible que el aumento en el monto del préstamo con el número de prestamistas pueda ser diferente para diferentes países y / o sectores. Probemos esto en el país RW. Por lo tanto, creamos nuevas variables que capturan esta interacción creando una nueva función de conteo de prestamistas para el país RW.

In [ ]:
# Interaction term for all countries other than RW
df['lc_country_others'] = np.where(~df['location_country_code'].isin(['RW']),df['lender_count'],0)
# Interaction term for Health
df['lc_country_rw'] = np.where(df['location_country_code'] == 'RW',df['lender_count'],0)

In [ ]:
# Define the dependent variable
y = df['loan_amount']
# Define the independent variables
X = df[['lender_count', 'country_RW', 'lc_country_rw']]
# Add an intercept term to the independent variables
X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model4 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model4.summary())

¡Mira eso! Con solo crear un término de interacción entre las características lender_count y location_country_code, hemos logrado aumentar nuestro R2 adj de 0.846 a 0.850. ¡No está mal! 

También mira los p-values nos dice que todos los coeficientes son significativos. La siguiente pregunta es, entonces, ¿cómo interpretamos los coeficientes de este término de interacción, lc_country_rw? Interpretamos esto de la siguiente manera; Si el país es Rwanda, por cada prestamista adicional, esperamos que el monto del préstamo disminuya en $ 14. El mismo concepto se puede aplicar a los otros coeficientes.

Ahora, ¿podemos agregar más variables a este modelo? Pensemos en los datos de préstamos y las variables de ingeniería para encontrar otras que podrían ser útiles. Agreguemos variables por sectores.

In [ ]:
for sect in df['sector'].unique():
    df['sector_'+sect] = np.where(df.sector ==sect, 1, 0)

In [ ]:
# Define the dependent variable
y = df['loan_amount']
# Define the independent variables
X = df[['lender_count', 'country_RW', 'lc_country_rw', 'sector_Education', 'sector_Clothing', 
        'sector_Personal Use']]
# Add an intercept term to the independent variables
X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model5 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model5.summary())

Agregar estas características ha mejorado ligeramente el modelo, el adj R2 ha aumentado de 0.850 a 0.853 y nuevamente, todos los p-valores son menores que 0.005, por lo que podemos considerarlo como significativo. Vamos a interpretar los resultados entonces ... 
- country_RW = 3513. En general, la gente en Rwanda solicita préstamos que son 3513 dólares más que el equivalente solicitado por personas en otros países africanos. 
- sector_Clothing = 155. En general, si un préstamo es para ropa, es 155 dólares más que otros. 
- sector_Educación = -442. En general, si un préstamo es para educación, es 442 dólares menos que otros. 
- etc etc

Podría intentar crear otras variables. Por ejemplo, podría generar atributos a partir de la variable "descripción". Por ejemplo, ¿cómo crearía una variable que indique si los niños se mencionan en la descripción?

<br>
<br>
<br>


** Ahora revisemos algunos de los supuestos de regresión lineal que cubrimos anteriormente para verificar que podemos interpretar con confianza los coeficientes de los modelos como se indica anteriormente **

Para hacer esto rápidamente, utilizaremos las estadísticas que figuran en la tabla de resumen. Para obtener más información sobre esto, consulte este [awesome page!](http://connor-johnson.com/2014/02/18/linear-regression-with-python/)

1) Normalidad: el número de Prob (JB) es inferior a 0,05, lo que significa que rechazamos la hipótesis nula de que los datos se distribuyen normalmente. **SUPUESTO NO CUMPLIDO**

2) Multicolinealidad: el número de condición es mayor que 30, lo que significa que tenemos un problema con la multicolinealidad en el modelo. **SUPUESTO NO CUMPLIDO**

3) Autocorrelación: el número de Durbin-Watson es aproximadamente 2, por lo que no hay autocorrelación. **SUPUESTO CUMPLIDO**

4) Homocedasticidad: el número de Prob (Omnibus) es menor que 0.05, lo que significa que rechazamos la Hipótesis nula de que los residuos están normalmente distribuidos. **SUPUESTO NO CUMPLIDO**

¡Claramente tenemos un problema aquí! Aunque el poder predictivo del modelo sigue siendo válido, lo que significa que todavía podemos predecir el uso del modelo con un R^2 de 0.853: __NO PODEMOS__ interpretar los coeficientes de la manera que esperamos.

El problema subyacente es que los datos no se distribuyen normalmente y que no siempre hay una relación lineal entre las variables dependientes e independientes. Por lo tanto, es posible que tengamos que mirar hacia modelos no paramétricos.

-----

Evaluación del Modelo
=====

Descartando el hecho de que nuestros supuestos de regresión lineal no se cumplen por un momento, vamos a evaluar el poder predictivo del modelo.

Para este propósito, vamos a cambiar al paquete de Regresión Lineal de sklearn. Esto nos permite utilizar otras métricas y funciones de sklearn para evaluar el rendimiento del modelo.

Una función importante que vamos a usar aquí es sklearn's [cross_val_score] (http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html)

Este modelo nos permite realizar una validación cruzada en nuestro conjunto de datos de entrenamiento y generar el score de nuestra elección. Veamos ...

In [ ]:
# Initialize the model. We set fit_intercept as false as we already have the constant included in our data. 
linear = LinearRegression(fit_intercept=False)

# Use the cross_val_sore function to calculate the r2 cross validation
r2_cross_val_score = cross_val_score(linear, X_train, y_train, cv = 5).mean()

# Results!
print('cross validation R2: {}'.format(r2_cross_val_score))

¡El puntaje R2 de validación cruzada es muy similar al R2 de entrenamiento, lo que nos da la confianza de que el modelo funcionará bien en los datos de prueba invisibles!

Otra métrica útil para evaluar el rendimiento de un modelo es el error cuadrático medio que nos dice cuánto error produjo nuestro modelo en promedio. ¡También podemos implementar esto en sklearn cross_val_score! De manera predeterminada, la función cross_val_score utilizará el puntaje predeterminado para el modelo, que en el caso de la regresión lineal es el R2.

Sin embargo, podemos 'crear' un anotador MSE de la siguiente manera y usarlo en su lugar ...

In [ ]:
# make MSE scorer
mse_scorer = make_scorer(mean_squared_error)
# use the cross_val_score function to calculate to the cross validated MSE
mse_cross_val_score = cross_val_score(linear, X_train, y_train, scoring=mse_scorer, cv =5).mean()
# Results!
print(f'cross validation MSE: {mse_cross_val_score:.6}')
print(f'cross validation RMSE: {np.sqrt(mse_cross_val_score):.4}')

Esto sugiere que el error promedio en nuestro modelo es de 951 dólares; esto parece un poco alto

** ¡Ahora veamos cómo funciona en nuestros datos de prueba! **

In [ ]:
#Train the Model
model2 = linear.fit(X_train, y_train)
#Get R2 for test data
print(f'R2 on test data: {model2.score(X_test, y_test):.4}')

¡Excelente! ¡Esto es comparable a la validación cruzada R2, por lo que sabemos que no estamos sobreajustando!

In [ ]:
# Get predictions for the test data
y_pred_test = model2.predict(X_test)
print(f'root mean squared error on test data: {np.sqrt(mean_squared_error(y_test,y_pred_test)):.4}')

Una vez más, esto es comparable a la puntuación validada cruzada, por lo que estamos seguros de que el modelo funciona bien en datos no vistos.

Finalmente, visualicemos cómo se desempeña nuestro modelo en los datos de prueba ...

In [ ]:
plt.figure(figsize=(9,7))
plt.scatter(y_pred_test, y_test, alpha=0.5, c='r')
plt.title('predicted vs true for test data')
plt.xlabel('predicted loan amounts')
plt.ylabel('True loan amounts')
plt.show();

-----
Appendix A.<br>Additional context on how to deal with skewed data:
----
<a id="skewed"> </a>


One potential solution is to **log transform** your data. For a quick review of logarithms, look [here](https://www.mathsisfun.com/algebra/logarithms.html). 

Note that when we log transform data, we change our interpretation of the final regression output. Previously we had a simple linear equation... 

    y = mx + b

This could be interpreted as, with every unit increase in x, we get a m increase in outcome feature y. Now we have a log equation: 

    log(y) = mx + b
    
This is less intuitive, as we have to solve for y. When we solve for y through exponentiation, we get the following equation: 

    y = 10^(mx + b)
    y = 10^(mx) * 10^(b)

The default base for log is 10. The takeaway here is that the impact x would have on y is much larger than linear - it is **exponentiated**. We will return to this interpretation in the discussion of the linear regression model. Let's try log transforming our loan_amount variable. 

In [ ]:
# Histogram of loan_amount 
plt.hist(df['loan_amount'])

In [ ]:
# Take the log of loan amount and plot histogram
log_loan_amount = np.log(df['loan_amount'])
plt.hist(log_loan_amount)

----
Appendix B <br> Choosing the right features
----

So we've increased our R2 by adding additional promising features from our Exploratory Data Analysis (EDA). However, how should we go about choosing the best combination of features for our model? In general there are two approachs:
- stepwise regression = Incrementally adding promising features and checking the R2 and p-valules at each step to check it is worth adding. This is the approach we used above. 
- Ridge or Lasso regression = These are methods which algorithmically reduce the beta coefficients for less important feature and can be used to find the most useful combination of features. More in depth information can be found here: [Regularization Regression](https://www.analyticsvidhya.com/blog/2016/01/complete-tutorial-ridge-lasso-regression-python/)

Let's test Lasso regression and see what we get! 

For context, Ridge and Lasso regression have a hyper-parameters called 'alpha' which determines how 'strong' the regularization affect is. 

A useful way to use Ridge or Lasso regression is to run the regression over a range of alphas and see which features maintain a large beta coefficient for the longest. It is these features which have the most predictive power!

In [ ]:
# Define the dependent variable
y = df['loan_amount']
# Define the independent variables
X = df[['lender_count', 'country_ZA', 'country_ZM', 'lc_country_rw', 'sector_Education', 'sector_Clothing', 
        'sector_Personal Use', 'sector_Retail', 'sector_Transportation', 'sector_Agriculture']]
# Add an intercept term to the independent variables
X['cnst'] = 1
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model6 = sm.OLS(endog = y_train,exog = X_train).fit()
print(model6.summary())

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
alphas = np.arange(0.001, 0.502, 0.002)
lasso_coefs = []
X_train_lasso= X_train[X_train.columns.tolist()] # Select columns / features for model

for a in alphas:
    lassoreg = Lasso(alpha=a, copy_X=True, normalize=True)
    lassoreg.fit(X_train_lasso, y_train)
    lasso_coefs.append(lassoreg.coef_)

In [ ]:
lasso_coefs = np.asarray(lasso_coefs).T

plt.figure(figsize=(14,10))
for coefs, feature in zip(lasso_coefs, X_train_lasso.columns):
    plt.plot(alphas, coefs, label = feature)
plt.legend(loc='best')
plt.show()

When alpha is approximately 0.25, the gold, green, and red lines go to 0